# Final Working

In [1]:
# Load Trained Model from dyslexia_model
from tensorflow import keras
model = keras.models.load_model('dyslexia_model')

In [4]:
# Basic Testing based on image [ available in dataset ]
import cv2
import numpy as np

# Read Actual Image
char = cv2.imread('Gambo/Train/Reversal/1_31975.png')
# Convert it to 28x28
char28x28 = cv2.resize(char, (28, 28))

# Pass converted image to model [ RGB Format ]
print("Prediction :: ",model.predict(np.array([char28x28])))

1/1 [==============================] - 1s 859ms/step
Prediction ::  [[0. 1.]]


In [64]:
# Actual Testing based on captured image
'''
    1. Read Handwriting image
    2. Convert it to grayscale
    3. Obtained it's Threshold
    4. Clean image [ morphological operations ]
    5. Detect color changes on image [ findContours ]
        5.1 : Iterate over each change
        5.2 : get exact character co-ordinates [ boundingRect ]
        5.3 : Crop character from actual image using co-ordinates
        5.4 : Convert cropped image into RGB 
        5.5 : Resize it to 28x28
        5.6 : store all characters in array [seperated_chars]
    6. Pass seperated_chars array to model for prediction
    7. Obtain mean of predicted array [ column ]
    8. (prediction_mean_arr[0] > prediction_mean_arr[1]) ? Not Dected : Dected
'''
# Load the image
img = cv2.imread('input/hw_05.jpg')

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Threshold the image to segment the characters
_, thresholded = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV)

# Perform morphological operations to clean up the image
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
thresholded = cv2.morphologyEx(thresholded, cv2.MORPH_CLOSE, kernel)

# Find the contours of the characters 
contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Iterate through the contours
seperated_chars = []
for i, contour in enumerate(contours):
    # Get the bounding rectangle of the contour
    x, y, w, h = cv2.boundingRect(contour)
    # Crop the character using array slicing
    character = thresholded[y:y+h, x:x+w]
    
    #convert each image into RGB format and pass it to model
    rgb = cv2.cvtColor(character, cv2.COLOR_GRAY2RGB)
    seperated_chars.append(cv2.resize(rgb, (28, 28)))
    
prediction_arr = model.predict(np.array(seperated_chars))
prediction_mean_arr = np.mean(prediction_arr, axis = 0)
print("Actual Prediction :: ",prediction_mean_arr)

if (prediction_mean_arr[0] > prediction_mean_arr[1]):
    print("Not Dected, accuracy: {:5.2f}%".format(100 * prediction_mean_arr[0]))
else:
    print("Dected, accuracy: {:5.2f}%".format(100 * prediction_mean_arr[1]))
    
cv2.imshow('Input Image', thresholded)
cv2.waitKey(0)
cv2.destroyAllWindows()

2/2 [==============================] - 0s 12ms/step
Actual Prediction ::  [0.36363637 0.6363636 ]
Dected, accuracy: 63.64%
